In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Load data into a pandas DataFrame
df_movies = pd.read_csv('https://raw.githubusercontent.com/Smobin916/-Fuzzy-Pandas-/refs/heads/main/movies_data.csv')

In [3]:
# Replace 'N/A' with NaN
df_movies['IMDB Rating'] = df_movies['IMDB Rating'].replace('N/A', pd.NA)

# Convert 'IMDB Rating' to float, coercing errors to NaN
df_movies['IMDB Rating'] = pd.to_numeric(df_movies['IMDB Rating'], errors='coerce')

# Drop rows with NaN values in 'IMDB Rating'
df_movies = df_movies.dropna(subset=['IMDB Rating'])

# Drop outliers / odd records
# display(df_movies[df_movies['Title'].str.contains('Lambs')])
df_movies = df_movies.drop(112)

# Drop duplicates
df_movies = df_movies.drop_duplicates()

# Sort by 'IMDB Rating' and keep the top 100 films
df_movies = df_movies.nlargest(100, 'IMDB Rating').reset_index(drop=True)

# Display the final DataFrame with top 100 movies
print(df_movies)

                       Title  Year Duration  IMDB Rating  \
0   The Shawshank Redemption  1994  142 min          9.3   
1              The Godfather  1972  175 min          9.2   
2            The Dark Knight  2008  152 min          9.0   
3           Schindler's List  1993  195 min          9.0   
4      The Godfather Part II  1974  202 min          9.0   
..                       ...   ...      ...          ...   
95                 Star Trek  2009  127 min          7.9   
96        The Breakfast Club  1985   97 min          7.8   
97  Ferris Bueller's Day Off  1986  103 min          7.8   
98                   Get Out  2017  104 min          7.8   
99         The Hateful Eight  2015  168 min          7.8   

                        Genre  \
0                       Drama   
1                Crime, Drama   
2        Action, Crime, Drama   
3   Biography, Drama, History   
4                Crime, Drama   
..                        ...   
95  Action, Adventure, Sci-Fi   
96             

In [4]:
# DATA CLEANUP AND PREP FOR EDA
# Create new columns for each genre, actor, and # of wins
df_movies[['Genre 1', 'Genre 2', 'Genre 3']] = df_movies['Genre'].str.split(',', expand=True)
df_movies[['Actor 1', 'Actor 2', 'Actor 3']] = df_movies['Actors'].str.split(',', expand=True)
df_movies[['Oscar wins', 'Total wins']] = df_movies['Wins'].str.split('.', expand=True)

# For 'Total wins', fill 'None' values with 0
df_movies['Total wins'] = pd.to_numeric(df_movies['Total wins'], errors='coerce')
df_movies.fillna({'Total wins': 0}, inplace=True)

# Convert 'Total wins' to integer
df_movies['Total wins'] = df_movies['Total wins'].astype(int)

# Display DataFrame
display(df_movies.head())

,Title,Year,Duration,IMDB Rating,Genre,Actors,Nominations,Wins,Popularity,Genre 1,Genre 2,Genre 3,Actor 1,Actor 2,Actor 3,Oscar wins,Total wins
0,The Shawshank Redemption,1994,142 min,9.3,Drama,"Tim Robbins, Morgan Freeman, Bob Gunton",42,Nominated for 7 Oscars. 21,"2,995,055",Drama,None,None,Tim Robbins,Morgan Freeman,Bob Gunton,Nominated for 7 Oscars,21
1,The Godfather,1972,175 min,9.2,"Crime, Drama","Marlon Brando, Al Pacino, James Caan",31,Won 3 Oscars. 31,"2,090,520",Crime,Drama,None,Marlon Brando,Al Pacino,James Caan,Won 3 Oscars,31
2,The Dark Knight,2008,152 min,9.0,"Action, Crime, Drama","Christian Bale, Heath Ledger, Aaron Eckhart",165,Won 2 Oscars. 164,"2,974,670",Action,Crime,Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Won 2 Oscars,164
3,Schindler's List,1993,195 min,9.0,"Biography, Drama, History","Liam Neeson, Ralph Fiennes, Ben Kingsley",49,Won 7 Oscars. 91,"1,501,081",Biography,Drama,History,Liam Neeson,Ralph Fiennes,Ben Kingsley,Won 7 Oscars,91
4,The Godfather Part II,1974,202 min,9.0,"Crime, Drama","Al Pacino, Robert De Niro, Robert Duvall",21,Won 6 Oscars. 17,"1,405,378",Crime,Drama,None,Al Pacino,Robert De Niro,Robert Duvall,Won 6 Oscars,17


In [33]:
# WHAT GENRES OF FILMS SHARE THE HIGHEST % PRESENT IN THE SELECTED TOP MOVIES?

# Get counts of each genre column
genre1_group = df_movies.groupby('Genre 1')['Title'].count().reset_index()
genre1_group.columns = ['Genre', 'Count']
genre2_group = df_movies.groupby('Genre 2')['Title'].count().reset_index()
genre2_group.columns = ['Genre', 'Count']
genre3_group = df_movies.groupby('Genre 3')['Title'].count().reset_index()
genre3_group.columns = ['Genre', 'Count']

display(genre1_group)
display(genre2_group)

test = pd.merge(genre1_group, genre2_group, on='Genre', how='outer', indicator=True)
display(test)

# test2 = genre1_group.merge(genre2_group, left_on='Genre', right_on='Genre', how='outer', indicator=True)
# display(test)

# test3 = pd.merge(pd.merge(genre1_group, genre2_group, on='Genre'), genre3_group, on='Genre', indicator=True)
# display(test3)

# test4 = pd.merge(genre1_group, genre2_group, left_index=True, right_index=True, how='outer', indicator=True)
# print(test4)

# test5 = pd.concat([genre1_group, genre2_group, genre3_group], axis=1).reset_index())

# test6 = [genre2_group, genre3_group]
# result = test6.join(dfs)

,Genre,Count
0,Action,24
1,Adventure,12
2,Animation,5
3,Biography,9
4,Comedy,9
5,Crime,13
6,Drama,24
7,Horror,4


,Genre,Count
0,Action,1
1,Adventure,20
2,Biography,1
3,Comedy,5
4,Crime,5
5,Drama,35
6,Family,2
7,Film-Noir,1
8,Horror,1
9,Music,1


,Genre,Count_x,Count_y,_merge
0,Action,NaN,1.0,right_only
1,Adventure,NaN,20.0,right_only
2,Biography,NaN,1.0,right_only
3,Comedy,NaN,5.0,right_only
4,Crime,NaN,5.0,right_only
5,Drama,NaN,35.0,right_only
6,Family,NaN,2.0,right_only
7,Film-Noir,NaN,1.0,right_only
8,Horror,NaN,1.0,right_only
9,Music,NaN,1.0,right_only


In [ ]:
# Create the initial DataFrame
df_actors = pd.DataFrame(df_movies)

# Split the 'Actors' column into lists
df_actors['Actors'] = df_actors['Actors'].str.split(', ')

# Explode the DataFrame to separate the actors into individual rows
df_actors = df_actors.explode('Actors')[['Actors']].reset_index(drop=True)

# Display the new DataFrame with individual actors
print(df_actors)

In [ ]:
# Alphabetize the DataFrame by 'Actors'
df_actors = df_actors.sort_values(by='Actors').reset_index(drop=True)

# Count duplicates for each actor
actor_counts = df_actors['Actors'].value_counts().reset_index()
actor_counts.columns = ['Actors', 'Count']

# Merge the counts back into the exploded DataFrame
df_actors = df_actors.merge(actor_counts, on='Actors')

# Drop duplicates
df_actors = df_actors.drop_duplicates().reset_index(drop=True)

# Display the final DataFrame
print(df_actors)

In [ ]:
# Sort by count in descending order and keep top 100 actors
top_actors = actor_counts.nlargest(100, 'Count').reset_index(drop=True)

# Display the final DataFrame with top 100 actors
print(top_actors)